In [ ]:
## Não se preocupem com essa célula. Ela é só pra essa aula funcionar no Google Colab
## Se estiver rodando isso no Colab, Descomente e rode o código abaixo:

#!git clone https://github.com/gavieira/curso_programacao

#import os
#os.chdir('/content/curso_programacao')
#!git pull
#!pip install biopython

# Biopython

## Importando módulos

**Módulos** são arquivos que contém funções/métodos geralmente não estão disponíveis por padrão no python. Os módulos podem ser carregados e usados no seu código (e te salvar muito tempo).

Para carregar um módulo, usamos a palavra-chave **import**:

In [4]:
# Importando módulo 'math'

import math
print(dir(math))
print(math.sqrt(25)) # Raiz quadrada
print(math.factorial(3)) # Fatorial

['__doc__', '__loader__', '__name__', '__package__', '__spec__', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'ceil', 'copysign', 'cos', 'cosh', 'degrees', 'e', 'erf', 'erfc', 'exp', 'expm1', 'fabs', 'factorial', 'floor', 'fmod', 'frexp', 'fsum', 'gamma', 'gcd', 'hypot', 'inf', 'isclose', 'isfinite', 'isinf', 'isnan', 'ldexp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'modf', 'nan', 'pi', 'pow', 'radians', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'tau', 'trunc']
5.0
6


Adicionamente, podemos usar duas palavras-chave combinadas com o **import**:

- **from**: Nos permite carregar apenas um ou alguns métodos/funções do módulo.
- **as**: Nos permite renomear o módulo o funções/métodos importados

In [ ]:
# Importando apenas método 'sqrt' do módulo 'math'

from math import sqrt   # E se adicionarmos o método 'factorial' ao import?

print(sqrt(25))
print(factorial(3))

In [ ]:
# Importando módulo 'math' com nome customizado

import math as matematica

print(matematica.sqrt(25))
print(matematica.factorial(3))

In [ ]:
# Importando métodos com nome customizado

from math import sqrt as raiz_quadrada, factorial as fatorial

print(raiz_quadrada(25))
print(fatorial(3))

## Manipulação de sequências de DNA sem módulos

Obviamente, como sequências de DNA são strings, podemos fazer várias operações sobre elas usando apenas os recursos básicos do Python:

In [ ]:
# Salvando uma sequência à variável "seq"
seq = "ATGGTATAA"

In [ ]:
# Imprimindo o tamanho da sequência
print("Essa sequencia tem {} nucleotídeos.".format(len(seq)))

In [ ]:
# Podemos pegar cada um dos nucleotídeos e o manusear como quisermos:
for nt in range(len(seq)):
    print("Index: {}: {}".format(nt, seq[nt]))

In [ ]:
# Podemos também separa a sequência em trincas (códons):
contador = 0
for nt in range(0, len(seq), 3):
    contador += 1
    print("Trinca {}: {}".format(contador, seq[nt:nt+3]))

Entretanto, há várias outras operações mais avançadas (como traduzir uma sequência de nucleotídeos em uma de aminoácidos) que são facilitadas por um módulo específico: o **Biopython**.

## Biopython

[**Biopython**](https://biopython.org/) é um módulo que possui várias ferramentas prontas para o manuseio de dados biológicos.

### Objeto `Seq`

O objeto mais simples em Biopython é o objeto Seq, que possui vários métodos interessantes:

In [ ]:
# Primeiro, vamos importar o que precisamos
from Bio.Seq import Seq

# E agora, criar nosso objeto Seq
dnaseq = Seq("ATGGTATAA")

print(type(dnaseq))
print(dir(dnaseq))

Com um objeto `Seq`, podemos, dentre outras coisas:

In [ ]:
# Gerar o reverso complementar da sequência
print("DNA original: {}".format(dnaseq))
print("Reverso complementar: {}".format(dnaseq.reverse_complement()))

In [ ]:
# Podemos transcrevê-la ou mesmo retrotranscrevê-la

rnaseq = dnaseq.transcribe()

print("RNA transcrito a partir desse DNA: {}".format(rnaseq))
print("Retrotranscrevendo o RNA: {}".format(rnaseq.back_transcribe()))

In [ ]:
# A tradução pode ser feita tanto a partir de DNA quanto de RNA

print("Proteína traduzida a partir de DNA: {}".format(dnaseq.translate()))
print("Proteína traduzida a partir de RNA: {}".format(rnaseq.translate()))

In [ ]:
# Podemos também obter o tamanho das sequências (DNA, RNA ou proteína)

# Gerando objeto com sequencia de aa
aaseq = dnaseq.translate()

print("Tamanho da sequência de DNA: {}".format(len(dnaseq)))
print("Tamanho da sequência de RNA: {}".format(len(rnaseq)))
print("Tamanho da sequência de proteína: {}".format(len(aaseq))) #OBS: Inclui stop codon!!

Será que conseguimos converter a informação da proteína para dna que nem na retrotranscrição do RNA? Vamos tentar!

In [ ]:
# Tentando retrotraduzir a sequência.
# Como não há um método back_translate(), iremos tentar com o back_transcribe()
print(aaseq.back_transcribe())

#### Não conseguimos retrotraduzir?

Por que deu erro?

O código genético é **degenerado**: Mais de um códon traduz para um mesmo aminoácido.

Ou seja, na ida da informação do DNA para proteína, só há uma tradução possível. Na volta para o DNA, há múltiplas.

### CodonTable

Para entender melhor a questão da degeneração do DNA, podemos acessar o código genético, que está disponível dentro do Biopython:

In [ ]:
from Bio.Data import CodonTable
print(CodonTable.unambiguous_dna_by_id.get(1)) # É um dicionário

#### Código genético é universal?

A maioria dos pesquisadores trabalham com genes codificados pelo Código Padrão (*Standard - Genetic Codon Table* 1). Mas há inúmeras variações do código dito "universal".

In [ ]:
for tabela in CodonTable.unambiguous_dna_by_id.values():
    print(tabela)

Até agora, nós brincamos somente com dados já presentes no Biopython ou com sequências curtas de DNA criadas na hora por nós mesmos.

Mas ao se trabalhar com bioinformática, precisamos ler arquivos que possuem sequências reais de organismos. Essas sequências geralmente são bem grandes (milhares de nt's) e podem possuir outras informações associadas (anotação de genes).

Vamos ver agora como trabalhar com sequências presentes em arquivos.

### SeqIO

Iremos agora aprender um pouco sobre o [**SeqIO**](https://biopython.org/wiki/SeqIO), um conjunto de ferramentas do Biopython que permite a leitura e `parsing` (extração das informações) dos mais diversos tipos de arquivos em bioinformática.

Aqui, focaremos em usar o **SeqIO para a análise de um arquivo *genbank***. Um arquivo genbank é muito mais complexo que um fasta, sendo dividido em 3 partes:

- **Header**: metadados sobre a sequência
- **Feature Table**: Anotações (no nosso caso, de genes)
- **Sequência**: Sequência de DNA

Vamos dar uma olhada nesse tipo de arquivo:

In [ ]:
# Abrindo arquivo genbank

with open("arquivos/pgracilis_mitocondria.gb", "r") as genbank:
    for line in genbank:
        print(line, end='')

Sendo um arquivo complexo, é mais difícil extrair informação de um genbank do que de um fasta, por exemplo. 

Mas o SeqIO consegue 'quebrar' o genbank em cada uma de suas informações ([*parsing* ou análise sintática](https://pt.wikipedia.org/wiki/An%C3%A1lise_sint%C3%A1tica_(computa%C3%A7%C3%A3o))), que podem ser acessadas individualmete. Isso facilita muito o uso desses arquivos.

Vamos ver isso com mais calma a seguir:

### `SeqIO`

Obviamente, para usar o SeqIO, precisamos primeiro importá-lo:

In [ ]:
from Bio import SeqIO

Para ler um arquivo genbank (ou qualquer outro arquivo suportado, na verdade), precisamos criar um **objeto SeqIO**. Para isso, precisamos usar o método `SeqIO.parse()`:

In [ ]:
genbank = SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank")
print(genbank)

Como podemos ver, esse objeto é na verdade um **gerador**. Um gerador é um **iterável**, ou seja, um tipo de dado no qual podemos usar loops para acessar todos os seus elementos, um por vez. Além disso, o gerador é **exaurível**, então iremos recriar o objeto toda vez que formos precisar dele.

Iterando sobre o objeto (que no caso possui apenas um elemento, já que o genbank continha apenas uma sequência), obtemos muita informação sobre o arquivo:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record)

Podemos notar que toda a informação já está categorizada em campos como `ID` ou `Description`. Ou seja, o método `SeqIO.parse()` faz o *parsing* do arquivo ao mesmo tempo em que cria o objeto SeqIO.

Note que esse objeto é muito mais complexo que o objeto `Seq`, o qual continha basicamente uma sequência de DNA, RNA ou proteína.

Vamos agora olhar para os **atributos** (logo falamos mais sobre o que é isso) e **métodos** desse objeto. Muitos desses nos permitem acesso direto aos dados do arquivo genbank:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(dir(record))

Vamos começar com o atributo `seq`, que contém a sequência nucleotídica do arquivo genbank:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record.seq)

#### Adendo: Atributos

Repare que na segunda linha da célula anterior

    print(record.seq)
    
`record.seq` não especifica um método, pois não tem os parênteses típicos da sintaxe de uma função/método. Em vez disso, ele especifica um **atributo**.

Da mesma forma que um método é uma função associada a um objeto, **um atributo é uma variável associada a um objeto**. Ela não modifica valores, apenas os guarda.

Se imprimirmos o atributo **"annotation"**, nós obteremos um **dicionário**, que pode ser usado para obter informações específicas:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record.annotations) # Não é um método
    print(type(record.annotations))

Se nós quisermos obter o nome da espécie, por exemplo, teremos que retirá-lo daí. "Organism" é uma chave nesse dicionário, e seu valor é o nome da espécie. Logo, podemos extrair essa informação usando o método `get()`:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record.annotations.get("organism"))

Nós também podemos usar a função `len()` no objeto SeqIO. Nesse caso, obteremos o tamanho do atributo `seq`. Ou seja, obteremos o tamanho da sequência.

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(len(record))
    print("O mitogenoma da formiga Pseudomyrmex gracilis possui {} nucleotídeos".format(len(record)))

Também podemos usar o método `format()` para converter o genbank em fasta:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record.format("fasta"))

Ou obter o reverso complementar da sequência com o método `reverse_complement()`:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(record.reverse_complement()) # Precisa usar o atributo seq
    print()
    print("Fita genbank:\n 5' {} 3'\n".format(record.seq[-10:])) # Fim da fita do genbank
    print("Reverso complementar:\n 5' {} 3'".format(record.reverse_complement().seq[:10])) # Começo da fita reversa

Com um pouco mais de esforço, podemos obter o reverso complementar em formato fasta:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(">{} (REVERSE COMPLEMENT)\n{}".format(record.description, record.seq[:500])) # Só os primeiros 500 nts

Além disso, podemos procurar por genes específicos dentro do arquivo genbank acessando o atributo `features` (que armazena uma lista de objetos chamados `objetos SeqFeature`):

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    print(type(record.features))
    print(record.features)

Com isso podemos, por exemplo, ver as informações associadas a todos os genes de rRNA presentes no genbank:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    for gene in record.features: # Há vários genes (features), cada um em um objeto SeqFeature
        if gene.type == "rRNA":
            print(gene)

Cada um dos objetos SeqFeature possui tanto o nome quanto a extensão dos genes. Sabendo isso, podemos imprimir todos os genes em formato fasta:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    for gene in record.features: # Há vários genes (features), cada um em um objeto SeqFeature
        if gene.type in ["rRNA", "CDS", "tRNA"]:
            cabecalho = gene.qualifiers.get('product')[0] # Extraindo nomes dos genes
            sequencia = gene.location.extract(record.seq) # Extraindo as sequências dos genes
            print(">{}\n{}\n".format(cabecalho, sequencia)) # Imprimindo o fasta na tela

Também podemos imprimir os genes como sequências de RNA, usando o método `transcribe()`:

In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    for gene in record.features: # Há vários genes (features), cada um em um objeto SeqFeature
        if gene.type in ["rRNA", "CDS", "tRNA"]:
            cabecalho = gene.qualifiers.get('product')[0] # Extraindo nomes dos genes
            sequencia = gene.location.extract(record.seq).transcribe() # Extraindo as sequências dos genes e transcrevendo-as
            print(">{}\n{}\n".format(cabecalho, sequencia)) # Imprimindo o fasta na tela

Por último, genes codificadores de proteínas (*Coding Sequences* ou CDS) podem ser traduzidos usando o método `translate()`:

> Bloco com recuo



In [ ]:
for record in SeqIO.parse("arquivos/pgracilis_mitocondria.gb", "genbank"):
    for gene in record.features: # Há vários genes (features), cada um em um objeto SeqFeature
        if gene.type == "CDS":
            cabecalho = gene.qualifiers.get('product')[0] # Extraindo nomes dos genes
            sequencia = gene.location.extract(record.seq).translate(5) # Extraindo as sequências dos genes e traduzindo para o código genético 5 (mitocondria de invertebrados)
            print(">{}\n{}\n".format(cabecalho, sequencia)) # Imprimindo o fasta na tela

Isso é o bastante sobre o SeqIO aplicado a arquivos genbank. Vale lembrar que o método `SeqIO.parse()` também pode ser usado para arquivos fasta:

In [ ]:
for record in SeqIO.parse("arquivos/multifasta.fasta", "fasta"):
    print(record) # Multifasta. Logo, há vários records diferentes
    print()

Vários dos métodos/funcionalidades encontrados para os arquivos genbank e objetos Seq também são válidos aqui, como: `len()`, `reverse_complement()`, `transcribe()` e `translate()`.

Isso é tudo, pessoal!

Espero que tenham curtido aprender um pouco sobre programação em si e como ela pode ser aplicada para lidar com dados biológicos.

Forte abraço e tudo de bom! ;)